Import Libraries

In [37]:
!pip install tweepy --upgrade
!pip install contractions
!pip install nltk
!pip install stop_words

import tweepy

import requests
import pandas as pd
import numpy as np

from google.colab import drive
drive.mount('/content/gdrive')
import sys
sys.path.insert(0,'/content/gdrive/My Drive/Colab Notebooks')
import config

import contractions
import nltk
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('omw-1.4')
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from stop_words import get_stop_words

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


In [38]:
client = tweepy.Client(config.bearer_token) #Insert twitter bearer token

Cleaning Tweets Function

In [ ]:
# Clean data function
def clean_tweets(tweets, depressed):

  # remove certain words only in tweets w/the word depression 
  if depressed == True:
    # Include tweets only containing the following regular expressions
    tweets = tweets.loc[tweets.tweet.str.contains("I\s|I'm|\sme\s|feel|felt|i'm|\smy\s"),:] 
  
  # Remove certain words in random tweets
  elif depressed == False:
    #Remove tweets indicating depression or anxiety
    tweets = tweets.loc[tweets.tweet.str.contains("""@|depress|anxiety""") == False,:]


  # Lowercase
  tweets['tweet'] = tweets['tweet'].str.lower()

  # Remove Contractions
  revised_tweet_list = []
  for tweet in tweets.tweet: 
    # create an empty list of expanded words
    expanded_words = []   
    for word in tweet.split():
      #  contractions.fix expands the words
      expanded_words.append(contractions.fix(word))  
    expanded_tweet = ' '.join(expanded_words)
    revised_tweet_list.append(expanded_tweet)
  tweets['expanded_tweet'] = revised_tweet_list

  # Remove Emojis
  remove_emojis = []
  for expanded_tweet in tweets.expanded_tweet:
    expanded_tweet=(expanded_tweet.encode('unicode-escape').decode('ASCII'))
    remove_emojis.append(expanded_tweet)
  tweets['tweet']= remove_emojis

  #Remove links, hashtag sign, ampersand, and UTF-8
  tweets = tweets[['tweet', 'id']] .replace({'tweet': {'\\.*|\\\\.*|https.*|amp;|#': ''}}, regex=True)

  #Remove short tweets (fewer than 25 characters)
  tweets = tweets[tweets['tweet'].str.len()>25]

  return(tweets)

##Get depressing tweets

*Depressed tweets are collected by entering 3 key words and its variants (depressed, antidepressants, anxiety) into our query*

In [39]:
# Select tweets with words depressed, antidepressants, and anxiety; exclude retweets, replies, and media
query_list = ["depressed lang:en -is:retweet -has:media -is:reply", "depression lang:en -is:retweet -has:media -is:reply", "antidepressants lang:en -is:retweet -has:media -is:reply", "antidepressant lang:en -is:retweet -has:media -is:reply", "anxiety lang:en -is:retweet -has:media -is:reply"]
tweets_list = []
id_list = []

for query in query_list:
  tweets = tweepy.Paginator(client.search_recent_tweets, query=query,
                              tweet_fields=['context_annotations', 'created_at'], max_results=100).flatten(limit=1200)
  for tweet in tweets:
      tweets_list.append(tweet.text)
      id_list.append(tweet.id)

df = pd.DataFrame(tweets_list, columns=['tweet'])
df['id'] = id_list
pd.options.display.max_colwidth = 300  # set a value as your need

### Manually remove irrelevant tweets

*Some tweets that depression or anxiety are used casually and don't have enough information to show that the user is depressed. Others use them to describe negative emotional experiences, even if is not an official diagnosis (i.e. hair depression, post-concert depression, etc). Others are used outside of our context of interest, such as the great depression. We manually remove these tweets*

In [33]:
df3 = df2.loc[df2.expanded_tweet.str.contains('nor am i depressed|tickets|he got depress|episode|does not give me the anxiety|cannot stop listening to|here are.*tips|album|totsuki|hopes it helps them understand|turns out i just needed|him into.*depression|ferrari|jimi hendrix|hair depress|hate a depressed|them snapchat memories|turns out i just needed|everyone seems depressed|was a depressed kid|he will inevitably get depressed|concert|kumbe|weyes blood|deftones|depressed people|shalin|veronica|depressed person|boobs are my|press a video|math anxiety|notifications|dogging the gym|who quoted this|posting or sharing|boob mri|anxiety eye color|hugs can have several benefits|less and less anxious|to bali|bocchi|anxiety money|never afraid of flying|seriously depressed me|the depressed one|thorfinn|salted caramel|taylor swift exists|research tells us|cough syrup as|yonc|hook up|@|cock|be depress|are depress|depression gone|depression nap|leaving my body|priyanka|eradicate my depression than|saved me from depression|shout out to|cure for depression|depression tweets|hot.*sexy|that was before the depression|great depression|post.* depression|my depression is cured|one with depression|cure[s]? my depression|depressed him|depressed her|is.*depress|you.*depress|my.*depressed|me depressed|not depressed|false alarm|my depressed|is depressed|be depress|depressed\?|follow|lakers|s[0-9]|watch|news|tv|show|having a sick dog|gender dysphoria|stressed and depressed|stole my antidepressants|going off antidepressants|".*"|basil|pharmaceutical compan|click here|works as an antidepressant|antidepressant lamp|clinical trial|antidepressant study|he best antidepressant i have|doki doki|okhty|bruce depression|ride out a depression|hair depression|will remain my antidepressant|teenage aggression|ball of anxiety') == False,:]

In [ ]:
#Get cleaned depressed tweets
df3 = clean_tweets(df, True)
df3.head()

,tweet,id
17,working from home today because i am depressed and making corporate small task at the office is the most energy taxing activity out of all my work projects,1623010166575706130
20,i been feeling down and depressed like i need one night to go skin out,1623010091174703104
32,i think i am depressed because i am not even excited for antman anymore,1623009514806181888
40,i am such a fucking extrovert that i get horribly depressed when i do not have regular interaction this is so tough,1623009130792329216
41,i am severely depressed and i think the only things that will help me out of my current state are a mall pretzel and a new tattoo,1623009040333778959


## Get non-depressing tweets w/word depression

In [ ]:
non_depressed = df2.loc[df2.expanded_tweet.str.contains('nor am i depressed|tickets|he got depress|cannot stop listening to|everyone seems depressed|was a depressed kid|he will inevitably get depressed|totsuki|anxiety eye color|here are.*tips|hopes it helps them understand|less and less anxious|cough syrup as|hugs can have several benefits|taylor swift exists|salted caramel|people prescribed antidepressants|him into.*depression|gender dysphoria|ferrari|great depression|hair depress')== True,:]
non_depressed = clean_tweets(non_depressed, True) # Clean data

In [ ]:
non_depressed.head()

,tweet,id
5,"well, we did each other's hair for a week then she got depressed and stopped doing dishes and now i hate her",1623010857834909701
9,"side note, i still do not have tickets for sza at td, so now i am even more depressed",1623010751056162842
12,i am not an angry nigga nor am i depressed but it is just something bout a mf saying goodmorning to me that piss me off,1623010595653005324
37,"she saw me depressed about renny tickets and dropped this , thank you queen",1623009398749806597
121,help me i do not think i am depressed but i cannot stop listening to aimee mann,1623004510108143618


##Get random tweets

In [47]:
# Get tweets
random_tweets_list = []
random_id_list = []

random_tweets = tweepy.Paginator(client.search_recent_tweets, query = "lang:en -is:retweet -has:media -the the",
                              tweet_fields=['context_annotations', 'created_at'], max_results=100).flatten(limit=6000)
for random_tweet in random_tweets:
    random_tweets_list.append(random_tweet.text)
    random_id_list.append(random_tweet.id)

rdf = pd.DataFrame(random_tweets_list, columns=['tweet'])
rdf['id'] = random_id_list
pd.options.display.max_colwidth = 300

rdf2 = clean_tweets(rdf, False)

<ipython-input-44-43bf30786194>:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tweets['tweet'] = tweets['tweet'].str.lower()
<ipython-input-44-43bf30786194>:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tweets['expanded_tweet'] = revised_tweet_list
<ipython-input-44-43bf30786194>:32: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-

In [23]:
len(rdf2)

2295

In [48]:
rdf2.head()

,tweet,id
0,i had like 8 alarms set and still overslept that is like letting a hail mary get completed,1623546544904609793
2,my burner page really be getting followers like ion even by tweeting shit just liking filth in peace,1623546544887836681
5,i am going to shit my mouth because yeen gon like what i got to say,1623546544850087937
9,you conducted yourself like a child today,1623546544774586371
24,have a day full of bing bong everyone!!,1623546544632008705


In [ ]:
# Add target variables to tweets
df3['depressed'] = 1
non_depressed['depressed'] = 0
rdf2['depressed'] = 0

# Export to CSV
df3.to_csv(r'depressed_tweets.csv')
non_depressed.to_csv(r'non_depressed.csv')
rdf2.to_csv(r'random_tweets.csv')